### Setup

In [1]:
# Import necessary libraries
from datetime import datetime, timedelta, timezone
import json
import os
import re
import random
import boto3
from time import sleep
from threading import Thread

import pandas as pd

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

from sagemaker.predictor import Predictor

from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import EndpointInput
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.model_monitor import CronExpressionGenerator

session = Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [2]:
# Helper functions
def invoke_endpoint(ep_name, file_name):
    with open(file_name, "r") as f:
        i = 0
        for row in f:
            payload = row.rstrip("\n")
            response = session.sagemaker_runtime_client.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType="text/csv",
                Body=payload,
                InferenceId=str(i),  # unique ID per row
            )["Body"].read()
            i += 1
            sleep(1)


def invoke_endpoint_forever():
    while True:
        try:
            invoke_endpoint(endpoint_name, "test_data/test-dataset-input-cols.csv")
        except session.sagemaker_runtime_client.exceptions.ValidationError:
            pass
        
def ground_truth_with_id(inference_id):
    random.seed(inference_id)  # to get consistent results
    rand = random.random()
    return {
        "groundTruthData": {
            "data": "1" if rand < 0.7 else "0",  # randomly generate positive labels 70% of the time
            "encoding": "CSV",
        },
        "eventMetadata": {
            "eventId": str(inference_id),
        },
        "eventVersion": "0",
    }


def upload_ground_truth(records, upload_time):
    fake_records = [json.dumps(r) for r in records]
    data_to_upload = "\n".join(fake_records)
    target_s3_uri = f"{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(fake_records)} records to", target_s3_uri)
    S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)
    
def generate_fake_ground_truth_forever():
    j = 0
    while True:
        fake_records = [ground_truth_with_id(i) for i in range(NUM_GROUND_TRUTH_RECORDS)]
        upload_ground_truth(fake_records, datetime.utcnow())
        j = (j + 1) % 5
        sleep(60 * 60)  # do this once an hour

In [3]:
# AWS region and IAM role
role = get_execution_role()
print("RoleArn:", role)

region = session.boto_region_name
print("Region:", region)

RoleArn: arn:aws:iam::004608622582:role/LabRole
Region: us-east-1


In [4]:
# Setup S3 bucket
bucket = session.default_bucket()
print("Demo Bucket:", bucket)
prefix = "sagemaker/Churn-ModelQualityMonitor-20201201"

##S3 prefixes
data_capture_prefix = f"{prefix}/datacapture"
s3_capture_upload_path = f"s3://{bucket}/{data_capture_prefix}"

ground_truth_upload_path = (
    f"s3://{bucket}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
)

reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket}/{reports_prefix}"

##Get the model monitor image
monitor_image_uri = image_uris.retrieve(framework="model-monitor", region=region)

print("Image URI:", monitor_image_uri)
print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

Demo Bucket: sagemaker-us-east-1-004608622582
Image URI: 156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer
Capture path: s3://sagemaker-us-east-1-004608622582/sagemaker/Churn-ModelQualityMonitor-20201201/datacapture
Ground truth path: s3://sagemaker-us-east-1-004608622582/sagemaker/Churn-ModelQualityMonitor-20201201/ground_truth_data/2024-06-07-17-24-54
Report path: s3://sagemaker-us-east-1-004608622582/sagemaker/Churn-ModelQualityMonitor-20201201/reports


In [5]:
# Upload test files
S3Uploader.upload("test_data/upload-test-file.txt", f"s3://{bucket}/test_upload")
print("Success! You are all set to proceed.")

Success! You are all set to proceed.


### Deploy pre-trained model with data capture

In [6]:
# Upload the pretrained model to S3
s3_key = f"s3://{bucket}/{prefix}"
model_url = S3Uploader.upload("model/xgb-churn-prediction-model.tar.gz", s3_key)
model_url

's3://sagemaker-us-east-1-004608622582/sagemaker/Churn-ModelQualityMonitor-20201201/xgb-churn-prediction-model.tar.gz'

In [7]:
# Create Sage Model Entity
model_name = f"DEMO-xgb-churn-pred-model-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"

image_uri = image_uris.retrieve(framework="xgboost", version="0.90-1", region=region)

model = Model(image_uri=image_uri, model_data=model_url, role=role, sagemaker_session=session)

In [8]:
# Deploy model with data capture
endpoint_name = f"DEMO-xgb-churn-model-quality-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"
print("EndpointName =", endpoint_name)

data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

EndpointName = DEMO-xgb-churn-model-quality-monitor-2024-06-07-1724
------!

In [9]:
# Create Sage Maker Predictor Object from the endpoint to be used to invoke the model
predictor = Predictor(
    endpoint_name=endpoint_name, sagemaker_session=session, serializer=CSVSerializer()
)

###  Generate a baseline for model quality performance

In [10]:
# Create churn and validate dataset
churn_cutoff = 0.8
validate_dataset = "validation_with_predictions.csv"

In [11]:
# Create probability, prediction, and labels
limit = 200  # Need at least 200 samples to compute standard deviations
i = 0
with open(f"test_data/{validate_dataset}", "w") as baseline_file:
    baseline_file.write("probability,prediction,label\n")  # our header
    with open("test_data/validation.csv", "r") as f:
        for row in f:
            (label, input_cols) = row.split(",", 1)
            probability = float(predictor.predict(input_cols))
            prediction = "1" if probability > churn_cutoff else "0"
            baseline_file.write(f"{probability},{prediction},{label}\n")
            i += 1
            if i > limit:
                break
            print(".", end="", flush=True)
            sleep(0.5)
print()
print("Done!")

........................................................................................................................................................................................................
Done!


In [12]:
# Examine predictions
!head test_data/validation_with_predictions.csv

probability,prediction,label
0.01516005303710699,0,0
0.1684480607509613,0,0
0.21427156031131744,0,0
0.06330718100070953,0,0
0.02791607193648815,0,0
0.014169521629810333,0,0
0.00571369007229805,0,0
0.10534518957138062,0,0
0.025899196043610573,0,0


In [13]:
# Create predictions for baseline dataset
baseline_prefix = prefix + "/baselining"
baseline_data_prefix = baseline_prefix + "/data"
baseline_results_prefix = baseline_prefix + "/results"

baseline_data_uri = f"s3://{bucket}/{baseline_data_prefix}"
baseline_results_uri = f"s3://{bucket}/{baseline_results_prefix}"
print(f"Baseline data uri: {baseline_data_uri}")
print(f"Baseline results uri: {baseline_results_uri}")

Baseline data uri: s3://sagemaker-us-east-1-004608622582/sagemaker/Churn-ModelQualityMonitor-20201201/baselining/data
Baseline results uri: s3://sagemaker-us-east-1-004608622582/sagemaker/Churn-ModelQualityMonitor-20201201/baselining/results


In [14]:
# Upload to baseline dataset
baseline_dataset_uri = S3Uploader.upload(f"test_data/{validate_dataset}", baseline_data_uri)
baseline_dataset_uri

's3://sagemaker-us-east-1-004608622582/sagemaker/Churn-ModelQualityMonitor-20201201/baselining/data/validation_with_predictions.csv'

In [15]:
# Create baseline job with validation datset predictions
# Create the model quality monitoring object
churn_model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=session,
)

In [16]:
# Name of the model quality baseline job
baseline_job_name = f"DEMO-xgb-churn-model-baseline-job-{datetime.utcnow():%Y-%m-%d-%H%M}"

In [17]:
# Execute the baseline suggestion job.

job = churn_model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    problem_type="BinaryClassification",
    inference_attribute="prediction",
    probability_attribute="probability",
    ground_truth_attribute="label",
)
job.wait(logs=False)

INFO:sagemaker:Creating processing-job with name DEMO-xgb-churn-model-baseline-job-2024-06-07-1730


................................................................................!

In [18]:
# Explore the results of the baseline job
baseline_job = churn_model_quality_monitor.latest_baselining_job

In [19]:
# View the metrics generated
binary_metrics = baseline_job.baseline_statistics().body_dict["binary_classification_metrics"]
pd.json_normalize(binary_metrics).T

,0
confusion_matrix.0.0,173
confusion_matrix.0.1,0
confusion_matrix.1.0,12
confusion_matrix.1.1,16
recall.value,0.571429
recall.standard_deviation,0.021816
precision.value,1.0
precision.standard_deviation,0.0
accuracy.value,0.940299
accuracy.standard_deviation,0.006087


In [20]:
# View the contraints generated
pd.DataFrame(baseline_job.suggested_constraints().body_dict["binary_classification_constraints"]).T

,threshold,comparison_operator
recall,0.571429,LessThanThreshold
precision,1.0,LessThanThreshold
accuracy,0.940299,LessThanThreshold
true_positive_rate,0.571429,LessThanThreshold
true_negative_rate,1.0,LessThanThreshold
false_positive_rate,0.0,GreaterThanThreshold
false_negative_rate,0.428571,GreaterThanThreshold
auc,0.939513,LessThanThreshold
f0_5,0.869565,LessThanThreshold
f1,0.727273,LessThanThreshold


### Setup continuous model monitoring to identify model quality drift

In [21]:
# Generate prediction data for model quality monitoring
thread = Thread(target=invoke_endpoint_forever)
thread.start()

In [22]:
# View Captured Data
print("Waiting for captures to show up", end="")
for _ in range(120):
    capture_files = sorted(S3Downloader.list(f"{s3_capture_upload_path}/{endpoint_name}"))
    if capture_files:
        capture_file = S3Downloader.read_file(capture_files[-1]).split("\n")
        capture_record = json.loads(capture_file[0])
        if "inferenceId" in capture_record["eventMetadata"]:
            break
    print(".", end="", flush=True)
    sleep(1)
print()
print("Found Capture Files:")
print("\n ".join(capture_files[-3:]))

Waiting for captures to show up...................................
Found Capture Files:
s3://sagemaker-us-east-1-004608622582/sagemaker/Churn-ModelQualityMonitor-20201201/datacapture/DEMO-xgb-churn-model-quality-monitor-2024-06-07-1724/AllTraffic/2024/06/07/17/28-28-727-394e4571-57b0-4661-b059-7992f825897b.jsonl
 s3://sagemaker-us-east-1-004608622582/sagemaker/Churn-ModelQualityMonitor-20201201/datacapture/DEMO-xgb-churn-model-quality-monitor-2024-06-07-1724/AllTraffic/2024/06/07/17/29-28-982-10bb0eb1-0bd0-4d5d-8fea-cf5de2fe893a.jsonl
 s3://sagemaker-us-east-1-004608622582/sagemaker/Churn-ModelQualityMonitor-20201201/datacapture/DEMO-xgb-churn-model-quality-monitor-2024-06-07-1724/AllTraffic/2024/06/07/17/37-03-445-ed96022a-9c6c-4611-bc5f-c5afc82cd976.jsonl


In [23]:
# Checkout the contents on a singular line
print("\n".join(capture_file[-3:-1]))

{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"105,0,125.4,116,261.5,95,241.6,104,11.4,9,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0.01716655306518078","encoding":"CSV"}},"eventMetadata":{"eventId":"fb922a5e-2118-4306-8d04-6343d0db2d44","inferenceId":"56","inferenceTime":"2024-06-07T17:38:01Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"61,0,260.0,123,210.5,127,234.7,70,9.0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0.5217539072036743","encoding":"CSV"}},"eventMetadata":{"eventId":"eacaa882-57e8-47f9-b144-2e3585e835

In [24]:
# View the json dump
print(json.dumps(capture_record, indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "186,0.1,137.8,97,187.7,118,146.4,85,8.7,6,1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,1.1,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.40,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50,0.51,0.52,0.53,1.2,1.3,0.54,1.4,0.55",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0.01584203727543354",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "0241be51-0e9f-4a26-8d70-3857555f051f",
    "inferenceId": "0",
    "inferenceTime": "2024-06-07T17:37:03Z"
  },
  "eventVersion": "0"
}


In [25]:
# Generate synthetic ground truth data
NUM_GROUND_TRUTH_RECORDS = 334  # 334 are the number of rows in data we're sending for inference

gt_thread = Thread(target=generate_fake_ground_truth_forever)
gt_thread.start()

Uploading 334 records to s3://sagemaker-us-east-1-004608622582/sagemaker/Churn-ModelQualityMonitor-20201201/ground_truth_data/2024-06-07-17-24-54/2024/06/07/17/3812.jsonl
Uploading 334 records to s3://sagemaker-us-east-1-004608622582/sagemaker/Churn-ModelQualityMonitor-20201201/ground_truth_data/2024-06-07-17-24-54/2024/06/07/18/3813.jsonl


In [26]:
# Create Monitoring Schedule
churn_monitor_schedule_name = (
    f"DEMO-xgb-churn-monitoring-schedule-{datetime.utcnow():%Y-%m-%d-%H%M}"
)

In [27]:
# Create an enpointInput
endpointInput = EndpointInput(
    endpoint_name=predictor.endpoint_name,
    probability_attribute="0",
    probability_threshold_attribute=0.5,
    destination="/opt/ml/processing/input_data",
)

In [47]:
# Create the monitoring schedule to execute every hour.
cron_expression = "cron(* * * * *)"

response = churn_model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=churn_monitor_schedule_name,
    endpoint_input=endpointInput,
    output_s3_uri=baseline_results_uri,
    problem_type="BinaryClassification",
    ground_truth_input=ground_truth_upload_path,
    constraints=baseline_job.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: DEMO-xgb-churn-monitoring-schedule-2024-06-07-1738


In [48]:
# Describing the schedule
churn_model_quality_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:004608622582:monitoring-schedule/DEMO-xgb-churn-monitoring-schedule-2024-06-07-1738',
 'MonitoringScheduleName': 'DEMO-xgb-churn-monitoring-schedule-2024-06-07-1738',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2024, 6, 7, 18, 0, 54, 978000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 6, 7, 18, 1, 2, 619000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2024-06-07-18-00-54-476',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'DEMO-xgb-churn-model-quality-monitor-2024-06-07-1724',
 'ResponseMetadata': {'RequestId': '0ea41bf2-d044-43bc-b2a9-75d51e8a2592',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0ea41bf2-d044-43bc-b2a9-75d51e8a2592',
   'content-type': 'application/x-amz-json-1.1',
   'conte

In [49]:
# Examine modelschedule execution
executions = churn_model_quality_monitor.list_executions()
executions

[]

In [50]:
# Wait for the first execution of the monitoring_schedule
print("Waiting for first execution", end="")
while True:
    execution = churn_model_quality_monitor.describe_schedule().get(
        "LastMonitoringExecutionSummary"
    )
    if execution:
        break
    print(".", end="", flush=True)
    sleep(10)
print()
print("Execution found!")

Waiting for first execution.................................................................................................................................................................................................................................................................................................................................................................................
Execution found!


In [51]:
while not executions:
    executions = churn_model_quality_monitor.list_executions()
    print(".", end="", flush=True)
    sleep(10)
latest_execution = executions[-1]
latest_execution.describe()

...........

{'ProcessingInputs': [{'InputName': 'groundtruth_input_1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-004608622582/sagemaker/Churn-ModelQualityMonitor-20201201/ground_truth_data/2024-06-07-17-24-54/2024/06/07/18',
    'LocalPath': '/opt/ml/processing/groundtruth/2024/06/07/18',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'endpoint_input_1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-004608622582/sagemaker/Churn-ModelQualityMonitor-20201201/datacapture/DEMO-xgb-churn-model-quality-monitor-2024-06-07-1724/AllTraffic/2024/06/07/18',
    'LocalPath': '/opt/ml/processing/input_data/DEMO-xgb-churn-model-quality-monitor-2024-06-07-1724/AllTraffic/2024/06/07/18',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputC

In [52]:
status = execution["MonitoringExecutionStatus"]

while status in ["Pending", "InProgress"]:
    print("Waiting for execution to finish", end="")
    latest_execution.wait(logs=False)
    latest_job = latest_execution.describe()
    print()
    print(f"{latest_job['ProcessingJobName']} job status:", latest_job["ProcessingJobStatus"])
    print(
        f"{latest_job['ProcessingJobName']} job exit message, if any:",
        latest_job.get("ExitMessage"),
    )
    print(
        f"{latest_job['ProcessingJobName']} job failure reason, if any:",
        latest_job.get("FailureReason"),
    )
    sleep(
        30
    )  # model quality executions consist of two Processing jobs, wait for second job to start
    latest_execution = churn_model_quality_monitor.list_executions()[-1]
    execution = churn_model_quality_monitor.describe_schedule()["LastMonitoringExecutionSummary"]
    status = execution["MonitoringExecutionStatus"]

print("Execution status is:", status)

if status != "Completed":
    print(execution)
    print(
        "====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures."
    )

Waiting for execution to finish.....................................................................!
groundtruth-merge-202406071900-d95d8ad7fa002d9f7f0b382e job status: Completed
groundtruth-merge-202406071900-d95d8ad7fa002d9f7f0b382e job exit message, if any: None
groundtruth-merge-202406071900-d95d8ad7fa002d9f7f0b382e job failure reason, if any: None
Waiting for execution to finish..........................................................................!
model-quality-monitoring-202406071900-d95d8ad7fa002d9f7f0b382e job status: Completed
model-quality-monitoring-202406071900-d95d8ad7fa002d9f7f0b382e job exit message, if any: CompletedWithViolations: Job completed successfully with 10 violations.
model-quality-monitoring-202406071900-d95d8ad7fa002d9f7f0b382e job failure reason, if any: None
Execution status is: CompletedWithViolations
{'MonitoringScheduleName': 'DEMO-xgb-churn-monitoring-schedule-2024-06-07-1738', 'ScheduledTime': datetime.datetime(2024, 6, 7, 19, 0, tzinfo=tzlocal(

In [53]:
latest_execution = churn_model_quality_monitor.list_executions()[-1]
report_uri = latest_execution.describe()["ProcessingOutputConfig"]["Outputs"][0]["S3Output"][
    "S3Uri"
]
print("Report Uri:", report_uri)

Report Uri: s3://sagemaker-us-east-1-004608622582/sagemaker/Churn-ModelQualityMonitor-20201201/baselining/results/DEMO-xgb-churn-model-quality-monitor-2024-06-07-1724/DEMO-xgb-churn-monitoring-schedule-2024-06-07-1738/2024/06/07/19


In [54]:
# View the violations
pd.options.display.max_colwidth = None
violations = latest_execution.constraint_violations().body_dict["violations"]
violations_df = pd.json_normalize(violations)
violations_df.head(10)

,constraint_check_type,description,metric_name
0,LessThanThreshold,Metric auc with 0.5126000805942783 +/- 0.004310506871509248 was LessThanThreshold '0.9395127993393898',auc
1,LessThanThreshold,Metric precision with 0.6950354609929078 +/- 0.01635440900746257 was LessThanThreshold '1.0',precision
2,LessThanThreshold,Metric truePositiveRate with 0.11493354182955434 +/- 0.001984016310886614 was LessThanThreshold '0.5714285714285714',truePositiveRate
3,LessThanThreshold,Metric f1 with 0.19724924521972492 +/- 0.0034900179550774574 was LessThanThreshold '0.7272727272727273',f1
4,LessThanThreshold,Metric accuracy with 0.322480181200453 +/- 0.003604860892477168 was LessThanThreshold '0.9402985074626866',accuracy
5,GreaterThanThreshold,Metric falsePositiveRate with 0.1324435318275154 +/- 0.007851215882209632 was GreaterThanThreshold '0.0',falsePositiveRate
6,LessThanThreshold,Metric trueNegativeRate with 0.8675564681724846 +/- 0.007851215882209633 was LessThanThreshold '1.0',trueNegativeRate
7,GreaterThanThreshold,Metric falseNegativeRate with 0.8850664581704457 +/- 0.0019840163108866163 was GreaterThanThreshold '0.4285714285714286',falseNegativeRate
8,LessThanThreshold,Metric recall with 0.11493354182955434 +/- 0.001984016310886614 was LessThanThreshold '0.5714285714285714',recall
9,LessThanThreshold,Metric f2 with 0.1379633974659784 +/- 0.0023956109936236025 was LessThanThreshold '0.625',f2


### Analyze model quality CloudWatch metrics

In [55]:
# Create CloudWatch client
cw_client = boto3.Session().client("cloudwatch")

namespace = "aws/sagemaker/Endpoints/model-metrics"

cw_dimensions = [
    {"Name": "Endpoint", "Value": endpoint_name},
    {"Name": "MonitoringSchedule", "Value": churn_monitor_schedule_name},
]

In [56]:
# List metrics through the pagination interface
paginator = cw_client.get_paginator("list_metrics")

for response in paginator.paginate(Dimensions=cw_dimensions, Namespace=namespace):
    model_quality_metrics = response["Metrics"]
    for metric in model_quality_metrics:
        print(metric["MetricName"])

recall
f2_best_constant_classifier
false_negative_rate
false_positive_rate
true_negative_rate
f2
precision_best_constant_classifier
recall_best_constant_classifier
f0_5_best_constant_classifier
f1
f0_5
true_positive_rate
total_number_of_violations
au_prc
accuracy
accuracy_best_constant_classifier
auc
precision
f1_best_constant_classifier


In [57]:
# Create Cloud Watch Alarm
alarm_name = "MODEL_QUALITY_F2_SCORE"
alarm_desc = (
    "Trigger an CloudWatch alarm when the f2 score drifts away from the baseline constraints"
)
mdoel_quality_f2_drift_threshold = (
    0.625  ##Setting this threshold purposefully low to see the alarm quickly.
)
metric_name = "f2"
namespace = "aws/sagemaker/Endpoints/model-metrics"

cw_client.put_metric_alarm(
    AlarmName=alarm_name,
    AlarmDescription=alarm_desc,
    ActionsEnabled=True,
    MetricName=metric_name,
    Namespace=namespace,
    Statistic="Average",
    Dimensions=[
        {"Name": "Endpoint", "Value": endpoint_name},
        {"Name": "MonitoringSchedule", "Value": churn_monitor_schedule_name},
    ],
    Period=600,
    EvaluationPeriods=1,
    DatapointsToAlarm=1,
    Threshold=mdoel_quality_f2_drift_threshold,
    ComparisonOperator="LessThanOrEqualToThreshold",
    TreatMissingData="breaching",
)

{'ResponseMetadata': {'RequestId': 'b1e1b5eb-a4e7-4e83-af21-88f29a027df0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b1e1b5eb-a4e7-4e83-af21-88f29a027df0',
   'content-type': 'text/xml',
   'content-length': '214',
   'date': 'Fri, 07 Jun 2024 19:18:08 GMT'},
  'RetryAttempts': 0}}

### Clean Up

In [59]:
churn_model_quality_monitor.delete_monitoring_schedule()
sleep(60)  # actually wait for the deletion

INFO:sagemaker:Deleting Monitoring Schedule with name: DEMO-xgb-churn-monitoring-schedule-2024-06-07-1738
INFO:sagemaker.model_monitor.model_monitoring:Deleting Model Quality Job Definition with name: model-quality-job-definition-2024-06-07-18-00-54-476


In [60]:
predictor.delete_model()
predictor.delete_endpoint()

INFO:sagemaker:Deleting model with name: sagemaker-xgboost-2024-06-07-17-24-56-242
INFO:sagemaker:Deleting endpoint configuration with name: DEMO-xgb-churn-model-quality-monitor-2024-06-07-1724
INFO:sagemaker:Deleting endpoint with name: DEMO-xgb-churn-model-quality-monitor-2024-06-07-1724


In [61]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>